# Pandas installation 

In [ ]:
%conda install pandas

# Dependencies libraries

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

notebook_dir = Path().resolve()
LAB_CODES_PATH = notebook_dir.parents[1] / "Data" / "laboratory_events_codes_2.csv"

read the `*.csv` files we need for the **DU**. 

In [9]:
df_2 = pd.read_csv(LAB_CODES_PATH)

Printing general information about the dataFrame `df_2`

In [38]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978503 entries, 0 to 978502
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   hadm_id            978503 non-null  float64
 1   charttime          978503 non-null  object 
 2   value              934025 non-null  object 
 3   valuenum           907317 non-null  float64
 4   valueuom           884585 non-null  object 
 5   ref_range_lower    865544 non-null  float64
 6   ref_range_upper    865544 non-null  float64
 7   flag               343687 non-null  object 
 8   label              978503 non-null  object 
 9   fluid              978503 non-null  object 
 10  examination_group  978503 non-null  object 
 11  analysis_batch_id  978503 non-null  object 
 12  qc_flag            978503 non-null  object 
 13  ref_range          146215 non-null  object 
dtypes: float64(4), object(10)
memory usage: 104.5+ MB


In [ ]:
print("Number of total records:", len(df_2))
# Numero di record con valore ref_range mancante
missing_ref_range = df_2['ref_range'].isnull().sum()
print("Number of records with missing ref_range values:", missing_ref_range)

Number of total records: 978503
Number of records with missing ref_range values: 832288


Il numero dei float corrispondono al numero di valori nulli che abbiamo nella colonna `ref_range`


In [ ]:
df_2['ref_range'].map(type).value_counts()

ref_range
<class 'float'>    832288
<class 'str'>      146215
Name: count, dtype: int64

# Missing values
We estimate the percentage of *missing values* contained in each column. 

In [ ]:
for col in df_2.columns:
    missing_count = df_2[col].isnull().sum()
    missing_percentage = (missing_count / len(df_2)) * 100
    print(f"Column '{col}': {missing_percentage:.2f}% missing values")

Now, we estimate the types hiddenly taken by each columns

In [ ]:
for col in df_2.columns:
    print (f"{col}: {df_2[col].map(type).value_counts()}")